In [2]:
import pandas as pd
import os
import sys

In [1]:
dim_cols_dict= {
                'item':{'key':['item_id'],
						'incols':['item_id','dept_id','cat_id'],
						'outcols':['item_id','dept_id','cat_id']
						},
               'store':{'key':['store_id'],
			            'incols':['store_id','state_id'],
						'outcols': ['store_id','state_id'] 
						}
              }

In [4]:
dim_cols_dict['item']

{'incols': ['item_id', 'dept_id', 'cat_id'],
 'key': ['item_id'],
 'outcols': ['item_id', 'dept_id', 'cat_id']}

In [29]:
def process_dimension(file_name,dim_cols_dict,dim_type):
  dim_name = 'dim_' + dim_type
  dim_key_cols = dim_cols_dict[dim_type]['key']
  dim_in_all_cols = dim_cols_dict[dim_type]['incols']
  dim_out_cols = dim_cols_dict[dim_type]['outcols']  
  file_path = 'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\\'
  #file_name = 'sales_train_validation.csv'
  dim_sk = dim_type + '_sk'
  dim_max_sk_csv = dim_name + '_max_sk.csv'
  dim_idx_lkp_csv = dim_name + '_idx_lkp.csv'
  dim_csv = dim_name + '.csv'
  dim_hash = dim_type + '_hash_key'
  dim_out_all_cols = list(dim_out_cols)
  dim_idx_cols = list(dim_key_cols)
  dim_out_all_cols.extend([dim_sk,dim_hash,'rec_hash'])
  dim_idx_cols.append(dim_sk)
  
  df1 = pd.read_csv(file_path + file_name )
  df1 = df1[dim_in_all_cols]
  df1.drop_duplicates(inplace=True)
  df1.reset_index(drop=True,inplace=True)
  try:
    df_max_lkp = pd.read_csv(file_path + dim_max_sk_csv)
    max_sk = df_max_lkp.iat[0,0]
  except:
    max_sk = 1
  try:
    df_dim_idx_lkp = pd.read_csv(file_path + dim_idx_lkp_csv,index_col=dim_hash)
  except:
    df_dim_idx_lkp = pd.DataFrame()
  
   
  def lookup_fun(x):
    try:
      return(df_dim_idx_lkp.at[x,dim_sk])
    except:
      return(-1)
  df1[dim_hash] = pd.util.hash_pandas_object(df1[dim_key_cols],index=False)
  df1['rec_hash'] = pd.util.hash_pandas_object(df1[dim_out_cols],index=False)
  df1[dim_sk] = df1[dim_hash].apply(lambda x: lookup_fun(x))
  df1_dim_updates = df1[df1[dim_sk] != -1]
  df1_dim_updates.reset_index(drop=True,inplace=True)
  df1_dim_inserts = df1[df1[dim_sk] == -1]
  if len(df1_dim_inserts) > 0:
    df1_dim_inserts.reset_index(drop=True,inplace=True)
    df1_dim_inserts[dim_sk] = df1_dim_inserts.index + max_sk
    max_sk = df1_dim_inserts[dim_sk].max() + 1
    if max_sk == max_sk:
      pd.DataFrame.from_dict({'max_sk':[max_sk]}).to_csv(file_path + dim_max_sk_csv,index=None)
    df1_dim_inserts.set_index(dim_hash,inplace=True)
    df_dim_idx_lkp = df_dim_idx_lkp.append(df1_dim_inserts[dim_idx_cols])
    df_dim_idx_lkp.to_csv(file_path + dim_idx_lkp_csv,header=True,index=True)
    df1_dim_inserts.reset_index(inplace=True)
  try:
    df1_dim_current = pd.read_csv(file_path + dim_csv)
  except:
    df1_dim_current = pd.DataFrame(columns=dim_out_all_cols)
  if len(df1_dim_updates) > 0:
    df1_dim_updates['check_dup_up'] = df1_dim_updates['rec_hash'].isin(df1_dim_current['rec_hash'])
    df1_dim_updates = df1_dim_updates[df1_dim_updates.check_dup_up == False]
    df1_dim_current['check_up'] = df1_dim_current['rec_hash'].isin(df1_dim_updates['rec_hash'])
    df1_dim_current = df1_dim_current[df1_dim_current.check_up == False]
    df1_dim_current = df1_dim_current[dim_out_all_cols].append(df1_dim_updates[dim_out_all_cols])
  df1_dim_current = df1_dim_current[dim_out_all_cols].append(df1_dim_inserts[dim_out_all_cols])
  df1_dim_current.to_csv(file_path + dim_csv ,header=True,index=None)
  return

In [7]:
file_name = 'sales_train_validation.csv'

In [30]:
process_dimension(file_name,dim_cols_dict,'item')

In [32]:
process_dimension(file_name,dim_cols_dict,'store')

In [25]:
file_path = 'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\\'

In [34]:
os.listdir('C:\Users\Rohan\Desktop\Kaggle\M5_Walmart')

['calendar.csv',
 'dim_item.csv',
 'dim_item_idx_lkp.csv',
 'dim_item_max_sk.csv',
 'dim_store.csv',
 'dim_store_idx_lkp.csv',
 'dim_store_max_sk.csv',
 'f_sales_trans_sk_mapped_test.csv',
 'm5-forecasting-accuracy.zip',
 'practice_sqls.sql',
 'sales_train_validation.csv',
 'sales_train_validation_10_items.csv',
 'sales_train_validation_10_items2.csv',
 'sales_train_validation_10_items_norm.csv',
 'sales_train_validation_mod1.csv',
 'sample_submission.csv',
 'sell_prices.csv',
 'walmart_heirarchy.csv',
 'walmart_star_schema.xlsx',
 '~$walmart_star_schema.xlsx']

In [27]:
os.listdir(file_path)

['calendar.csv',
 'm5-forecasting-accuracy.zip',
 'practice_sqls.sql',
 'sales_train_validation.csv',
 'sales_train_validation_10_items.csv',
 'sales_train_validation_10_items2.csv',
 'sales_train_validation_10_items_norm.csv',
 'sales_train_validation_mod1.csv',
 'sample_submission.csv',
 'sell_prices.csv',
 'walmart_heirarchy.csv',
 'walmart_star_schema.xlsx',
 '~$walmart_star_schema.xlsx']

In [28]:
file_path

'C:\\Users\\Rohan\\Desktop\\Kaggle\\M5_Walmart\\'